<a href="https://colab.research.google.com/github/prabhhhneet/Gradient-Descent-python/blob/main/mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# Data Preprocessing 
# As the images are of different colour, different sizes. So in this the 
# main aim is to convert the images into grey and resize into 100*100
import cv2,os

data_path='/content/drive/MyDrive/dataset'
# load folder names
categories=os.listdir(data_path)
# number of folders
labels=[i for i in range(len(categories))]

# dictionary for labels
label_dict=dict(zip(categories,labels)) #empty dictionary

print(label_dict)
print(categories)
print(labels)

{'without_mask': 0, 'with_mask': 1}
['without_mask', 'with_mask']
[0, 1]


In [17]:
# size of image
img_size=100
#empty label lists to store the images
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(gray,(img_size,img_size))
            #resizing the gray scale into 100x100, since we need a fixed common
            # size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
 #normalize the images
import numpy as np

#to convert it into pixel range(0,1)
data=np.array(data)/255.0

#to convert 4d array
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils import np_utils

# as last layer of neural network has 2 neurons so the target should be
# converted to two values(0,1) 
new_target=np_utils.to_categorical(target)

In [20]:
# save the data and target
np.save('data',data)
np.save('target',new_target)

In [21]:
import numpy as np

data=np.load('data.npy')
target=np.load('target.npy')

#loading the save numpy arrays in the previous code

In [22]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [23]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [ ]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/20
31/31 [==============================] - 81s 3s/step - loss: 0.6375 - accuracy: 0.6331 - val_loss: 0.5059 - val_accuracy: 0.8145
Epoch 2/20
31/31 [==============================] - 79s 3s/step - loss: 0.3812 - accuracy: 0.8417 - val_loss: 0.2967 - val_accuracy: 0.8911
Epoch 3/20
31/31 [==============================] - 77s 3s/step - loss: 0.2231 - accuracy: 0.9062 - val_loss: 0.3263 - val_accuracy: 0.8468
Epoch 4/20
31/31 [==============================] - 78s 3s/step - loss: 0.1649 - accuracy: 0.9466 - val_loss: 0.2530 - val_accuracy: 0.8952
Epoch 5/20
31/31 [==============================] - 79s 3s/step - loss: 0.1565 - accuracy: 0.9385 - val_loss: 0.2107 - val_accuracy: 0.9315
Epoch 6/20
31/31 [==============================] - 77s 2s/step - loss: 0.1165 - accuracy: 0.9627 - val_loss: 0.2550 - val_accuracy: 0.9032
Epoch 7/20
31/31 [==============================] - 79s 3s/step - loss: 0.0932 - accuracy: 0.9667 - val_loss: 0.1824 - val_accuracy: 0.9234
Epoch 8/20
31/31 [==

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'],'r',label='training loss')
plt.plot(history.history['val_loss'],label='validation loss')
plt.xlabel('# epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'],'r',label='training accuracy')
plt.plot(history.history['val_accuracy'],label='validation accuracy')
plt.xlabel('# epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
print(model.evaluate(test_data,test_target))

In [ ]:
from keras.models import load_model
import cv2
import numpy as np

In [ ]:
model = load_model('drive/MyDrive/dataset/haarcascade/model-017.model')

face_clsfr=cv2.CascadeClassifier('drive/MyDrive/dataset/haarcascade/haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(2)

labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [ ]:
while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()
